In [2]:
!pip install --no-input earthaccess h5netcdf xarray "dask[complete]" matplotlib cartopy

In [9]:
#!/usr/bin/env python3
"""
PACE OCI chlorophyll: rolling 8-day median composites + change maps
Saves into a date-specific folder: <base>/figs/YYYYMMDD/

Outputs (in that folder):
  1) PACE_CHL_last8d_median_YYYYMMDD.png
  2) PACE_CHL_prev8d_median_YYYYMMDD.png
  3) PACE_CHL_change_last8d_vs_prev_YYYYMMDD.png
  4) PACE_CHL_pct_change_last8d_vs_prev_YYYYMMDD.png
Optional: NetCDF for current composite.

Robust to 0..360 vs -180..180 lon, descending lats, sparse data, NRT quirks.
"""

import os
from datetime import datetime, timedelta, timezone
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

# Cartopy (optional, recommended)
try:
    import cartopy.crs as ccrs
    import cartopy.feature as cfeature
    from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
    import matplotlib.ticker as mticker
    HAS_CARTOPY = True
except Exception:
    HAS_CARTOPY = False

# Notebook "Download →" link (if on remote)
try:
    from IPython.display import display, FileLink
    IN_NOTEBOOK = True
except Exception:
    IN_NOTEBOOK = False

import earthaccess

# ====================== USER SETTINGS ======================
BBOX_INPUT = (-58.623047, -43.516689, -37.968750, -31.728167)     # (W, S, E, N)
NDAYS = 8
RES_TAG = "0p1deg"                                                # inside granule name
VMIN, VMAX = None, None                                           # e.g., (0.05, 5.0)
WRITE_NETCDF = False

# Local base path on your Mac (used automatically if exists)
PREFERRED_LOCAL_BASE = "/Users/nicholasbaetge/github/cruise_context/SPARC_2025/SPARC_cruise_context_git"
SUBDIR_ROOT = "figs"  # everything will go under <base>/<SUBDIR_ROOT>/<YYYYMMDD>/

# Fallback base (e.g., CryoCloud)
OUTDIR_FALLBACK_BASE = "./"
# ===========================================================


# ----------------- Helpers -----------------
def utc_datestring(dt):
    return dt.strftime("%Y-%m-%d")

def last_n_days_utc(n):
    end = datetime.now(timezone.utc).date() - timedelta(days=1)      # end = yesterday
    start = end - timedelta(days=n-1)
    return utc_datestring(start), utc_datestring(end)

def window_before(t0_str, n):
    t0 = datetime.fromisoformat(t0_str).date()
    prev_end = t0 - timedelta(days=1)
    prev_start = prev_end - timedelta(days=n-1)
    return prev_start.strftime("%Y-%m-%d"), prev_end.strftime("%Y-%m-%d")

def pick_chl_var(ds):
    candidates = ["chlor_a", "chl", "CHL", "chlor"]
    for v in ds.data_vars:
        if any(k in v.lower() for k in candidates):
            return v
    if "chlor_a" in ds:
        return "chlor_a"
    raise KeyError("Could not find a chlorophyll variable in dataset.")

def get_lonlat_names(da):
    lon_name = next((n for n in ["lon", "longitude", "x"] if n in da.coords), None)
    lat_name = next((n for n in ["lat", "latitude", "y"] if n in da.coords), None)
    if lon_name is None or lat_name is None:
        raise KeyError("Could not find lon/lat coordinate names.")
    return lon_name, lat_name

def convert_bbox_to_range(w, s, e, n, lon_min, lon_max):
    """Project bbox to dataset's lon range (either -180..180 or 0..360)."""
    want_0360 = (lon_min >= 0) and (lon_max > 180)
    want_m180 = (lon_min < 0) and (lon_max <= 180)
    if want_0360:
        to0360 = lambda x: x % 360 if (x % 360) >= 0 else (x % 360) + 360
        return (to0360(w), s, to0360(e), n), "0360"
    elif want_m180:
        to_m180 = lambda x: ((x + 180) % 360) - 180
        return (to_m180(w), s, to_m180(e), n), "m180"
    else:
        return (w, s, e, n), "as_is"

def slice_with_lat_any_order(da, lat_name, south, north):
    lat_vals = da[lat_name]
    ascending = bool(lat_vals[0] < lat_vals[-1])
    return da.sel({lat_name: slice(south, north)}) if ascending else da.sel({lat_name: slice(north, south)})

def robust_percentiles(arr, p1=2, p2=98, fallback=(0.05, 5.0)):
    finite = np.isfinite(arr)
    if finite.sum() < 10:
        return fallback
    try:
        vmin = float(np.nanpercentile(arr[finite], p1))
        vmax = float(np.nanpercentile(arr[finite], p2))
        if not np.isfinite(vmin) or not np.isfinite(vmax) or vmin >= vmax:
            return fallback
        return vmin, vmax
    except Exception:
        return fallback

def make_outdir(date_tag):
    """Create a date-specific folder under the best available base dir."""
    if os.path.exists(PREFERRED_LOCAL_BASE):
        base = PREFERRED_LOCAL_BASE
    else:
        base = OUTDIR_FALLBACK_BASE
    outdir = os.path.join(base, SUBDIR_ROOT, date_tag)
    os.makedirs(outdir, exist_ok=True)
    return outdir

def search_open_composite(t0, t1, bbox, res_tag, short_name="PACE_OCI_L3M_CHL_NRT"):
    results = earthaccess.search_data(
        short_name=short_name,
        temporal=(t0, t1),
        bounding_box=bbox,
        granule_name=f"*.DAY.*.{res_tag}.*",
        cloud_hosted=True,
    )
    if not results:
        return None, None
    paths = earthaccess.open(results[-NDAYS:])  # last N
    ds = xr.open_mfdataset(
        paths, combine="nested", concat_dim="date",
        engine="h5netcdf", mask_and_scale=True, decode_cf=True, chunks="auto"
    )
    vname = pick_chl_var(ds)
    field = ds[vname].median(dim="date", skipna=True)
    return ds, field

def subset_to_bbox(field, bbox_input):
    lon_name, lat_name = get_lonlat_names(field)
    ds_lon_min = float(field[lon_name].min())
    ds_lon_max = float(field[lon_name].max())
    (w, s, e, n), mode = convert_bbox_to_range(*bbox_input, ds_lon_min, ds_lon_max)
    crosses_dateline = (e < w)
    if crosses_dateline:
        left = field.sel({lon_name: slice(w, ds_lon_max)})
        right = field.sel({lon_name: slice(ds_lon_min, e)})
        subset_lon = xr.concat([left, right], dim=lon_name)
    else:
        subset_lon = field.sel({lon_name: slice(w, e)})
    subset = slice_with_lat_any_order(subset_lon, lat_name, s, n)
    if subset.size == 0 or subset.shape[-2] == 0 or subset.shape[-1] == 0:
        raise RuntimeError("Empty subset—bbox may not overlap data.")
    meta = dict(lon_name=lon_name, lat_name=lat_name, w=w, e=e, s=s, n=n,
                ds_lon_min=ds_lon_min, ds_lon_max=ds_lon_max, crosses=crosses_dateline)
    return subset, meta

def draw_map(ax, data, meta, vmin, vmax, title, cmap="viridis"):
    if not HAS_CARTOPY:
        im = ax.imshow(
            data.values,
            origin="lower",
            vmin=vmin, vmax=vmax, cmap=cmap,
            extent=[float(data[meta["lon_name"]].min()), float(data[meta["lon_name"]].max()),
                    float(data[meta["lat_name"]].min()), float(data[meta["lat_name"]].max())]
        )
        cb = plt.colorbar(im, ax=ax, fraction=0.035, pad=0.03)
        cb.set_label("Chlorophyll-a (mg m$^{-3}$)")
        ax.set_xlabel("Longitude"); ax.set_ylabel("Latitude")
        ax.set_title(title, fontsize=11)
        return

    crs = ccrs.PlateCarree()
    im = ax.pcolormesh(
        data[meta["lon_name"]], data[meta["lat_name"]], data,
        transform=crs, shading="auto", vmin=vmin, vmax=vmax, cmap=cmap
    )
    ax.coastlines(resolution="10m", linewidth=0.6)
    ax.add_feature(cfeature.LAND, facecolor="0.85", zorder=0)
    xe = meta["e"] if not meta["crosses"] else (meta["w"] + meta["e"]) / 2
    ax.set_extent([meta["w"], xe, meta["s"], meta["n"]], crs=crs)
    # ticks/labels
    x_ticks = np.linspace(min(meta["w"], meta["e"]), max(meta["w"], meta["e"]), 5)
    y_ticks = np.linspace(meta["s"], meta["n"], 5)
    ax.set_xticks(x_ticks, crs=crs); ax.set_yticks(y_ticks, crs=crs)
    ax.xaxis.set_major_formatter(LongitudeFormatter(number_format=".1f", degree_symbol="°"))
    ax.yaxis.set_major_formatter(LatitudeFormatter(number_format=".1f", degree_symbol="°"))
    gl = ax.gridlines(crs=crs, linewidth=0.3, color="gray", alpha=0.5, linestyle="--", draw_labels=False)
    gl.xlocator = mticker.MaxNLocator(5); gl.ylocator = mticker.MaxNLocator(5)
    cb = plt.colorbar(im, ax=ax, fraction=0.035, pad=0.03)
    cb.set_label("Chlorophyll-a (mg m$^{-3}$)")
    ax.set_title(title, fontsize=11)

def save_png(fig, outdir, fname, link=True):
    path = os.path.join(outdir, fname)
    plt.tight_layout(); plt.savefig(path, dpi=200); plt.close(fig)
    print(f"Saved: {path}")
    # Offer a download link when running on remote
    if IN_NOTEBOOK and (not os.path.exists(PREFERRED_LOCAL_BASE)) and link:
        display(FileLink(path, result_html_prefix="Download → "))

# ----------------- Main -----------------
def main():
    earthaccess.login(persist=True)

    # Windows (current & previous)
    t0, t1 = last_n_days_utc(NDAYS)
    t0_prev, t1_prev = window_before(t0, NDAYS)
    date_tag = t1.replace("-", "")  # e.g., 20251104

    # Create date-specific output dir: <base>/figs/YYYYMMDD/
    outdir = make_outdir(date_tag)

    # Current window composite
    ds_cur, field_cur = search_open_composite(t0, t1, BBOX_INPUT, RES_TAG)
    if field_cur is None:
        raise RuntimeError("No current-window granules found.")
    subset_cur, meta = subset_to_bbox(field_cur, BBOX_INPUT)

    # Previous window composite
    ds_prev, field_prev = search_open_composite(t0_prev, t1_prev, BBOX_INPUT, RES_TAG)
    subset_prev = None
    if field_prev is not None:
        subset_prev, _ = subset_to_bbox(field_prev, BBOX_INPUT)

    # Color limits for composites
    if VMIN is None or VMAX is None:
        vmin_cur, vmax_cur = robust_percentiles(subset_cur.values, 2, 98, fallback=(0.05, 5.0))
        if subset_prev is not None:
            vmin_prev, vmax_prev = robust_percentiles(subset_prev.values, 2, 98, fallback=(0.05, 5.0))
        else:
            vmin_prev, vmax_prev = vmin_cur, vmax_cur
    else:
        vmin_cur = vmin_prev = VMIN
        vmax_cur = vmax_prev = VMAX

    # 1) Current 8-day median
    title = f"PACE OCI L3M CHL median (last {NDAYS} days): {t0} → {t1}"
    fig, ax = (plt.subplots(figsize=(8,7), subplot_kw={"projection": ccrs.PlateCarree()})
               if HAS_CARTOPY else plt.subplots(figsize=(8,7)))
    draw_map(ax, subset_cur, meta, vmin_cur, vmax_cur, title, cmap="viridis")
    save_png(fig, outdir, f"PACE_CHL_last{NDAYS}d_median_{date_tag}.png")

    # 2) Previous 8-day median (if available)
    if subset_prev is not None:
        title_prev = f"PACE OCI L3M CHL median (prev {NDAYS} days): {t0_prev} → {t1_prev}"
        fig, ax = (plt.subplots(figsize=(8,7), subplot_kw={"projection": ccrs.PlateCarree()})
                   if HAS_CARTOPY else plt.subplots(figsize=(8,7)))
        draw_map(ax, subset_prev, meta, vmin_prev, vmax_prev, title_prev, cmap="viridis")
        save_png(fig, outdir, f"PACE_CHL_prev{NDAYS}d_median_{date_tag}.png")
    else:
        print("No previous-window granules found; skipping previous composite & change maps.")

    # 3) Change maps (Δ and %Δ)
    if subset_prev is not None:
        delta = subset_cur - subset_prev
        with np.errstate(divide="ignore", invalid="ignore"):
            pct_change = 100.0 * (delta / subset_prev)
        pct_change = pct_change.where(np.isfinite(pct_change) & (np.abs(pct_change) < 1000))

        abs95 = np.nanpercentile(np.abs(delta.values[np.isfinite(delta.values)]), 95) if np.isfinite(delta).any() else 0.5
        rng = float(abs95) if np.isfinite(abs95) and abs95 > 0 else 0.5

        # Δ (mg m^-3)
        fig, ax = (plt.subplots(figsize=(8,7), subplot_kw={"projection": ccrs.PlateCarree()})
                   if HAS_CARTOPY else plt.subplots(figsize=(8,7)))
        draw_map(ax, delta, meta, -rng, +rng,
                 f"Δ CHL (current 8d − previous 8d)\n[{t0}→{t1}] minus [{t0_prev}→{t1_prev}]",
                 cmap="RdBu_r")
        save_png(fig, outdir, f"PACE_CHL_change_last{NDAYS}d_vs_prev_{date_tag}.png")

        # %Δ (clip to ±100%)
        fig, ax = (plt.subplots(figsize=(8,7), subplot_kw={"projection": ccrs.PlateCarree()})
                   if HAS_CARTOPY else plt.subplots(figsize=(8,7)))
        draw_map(ax, pct_change, meta, -100, +100,
                 f"%Δ CHL (current vs previous 8d)\n[{t0}→{t1}] vs [{t0_prev}→{t1_prev}]",
                 cmap="RdBu_r")
        save_png(fig, outdir, f"PACE_CHL_pct_change_last{NDAYS}d_vs_prev_{date_tag}.png")

    # Optional NetCDF for current composite
    if WRITE_NETCDF:
        nc_path = os.path.join(outdir, f"PACE_CHL_last{NDAYS}d_median_{date_tag}.nc")
        subset_cur.to_dataset(name="chlor_a_median").to_netcdf(nc_path)
        print(f"Saved composite NetCDF: {nc_path}")
        if IN_NOTEBOOK and (not os.path.exists(PREFERRED_LOCAL_BASE)):
            display(FileLink(nc_path, result_html_prefix="Download → "))

    # Close datasets
    try:
        if ds_cur is not None:
            ds_cur.close()
        if ds_prev is not None:
            ds_prev.close()
    except NameError:
        pass


# Run
if __name__ == "__main__":
    main()


QUEUEING TASKS | :   0%|          | 0/8 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/8 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/8 [00:00<?, ?it/s]

QUEUEING TASKS | :   0%|          | 0/8 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/8 [00:00<?, ?it/s]

...

/home/jovyan/cruise_context/SPARC/figs/20251104/PACE_CHL_prev8d_median_20251104.png

Saved: ./figs/20251104/PACE_CHL_change_last8d_vs_prev_20251104.png


/home/jovyan/cruise_context/SPARC/figs/20251104/PACE_CHL_change_last8d_vs_prev_20251104.png

Saved: ./figs/20251104/PACE_CHL_pct_change_last8d_vs_prev_20251104.png


/home/jovyan/cruise_context/SPARC/figs/20251104/PACE_CHL_pct_change_last8d_vs_prev_20251104.png